In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from ase.io import read

In [2]:
zro2_files = os.listdir('../2_zr_o2/Dataset_ZrO2')

In [3]:
def local_enviroment(mol):
    from ase import Atoms
    from ase.build import molecule
    from ase.neighborlist import NeighborList

    cutoff = 2
    cutoffs = [cutoff] * len(mol)
    nl = NeighborList(cutoffs, self_interaction=False, bothways=True)
    nl.update(mol)
    clusters = []
    visited = set()

    for i in range(len(mol)):
        if i not in visited:
            indices, _ = nl.get_neighbors(i)
            cluster = set(indices) | {i}
            clusters.append(cluster)
            visited.update(cluster)

    # print(f"Número de estruturas dentro do cutoff {cutoff} Å: {len(clusters)}")
    # for idx, cluster in enumerate(clusters):
    #     print(f"Cluster {idx + 1}: {[mol.symbols[i] for i in cluster]}")
    
    return clusters

In [13]:
dictionary_zro2 = {}
sum = 0

for file in tqdm(range(len(zro2_files))):

    structures = read(f'../2_zr_o2/Dataset_ZrO2/{zro2_files[file]}',index=':')

    for structure in structures:
        
        clusters = local_enviroment(structure)
        local_env,lclusters = 0,len(clusters)
        
        for i in range(lclusters): local_env += len(clusters[i])
        local_env = local_env/lclusters

        dictionary_zro2[sum] = [zro2_files[file],local_env]
        sum+=1

100%|██████████| 35/35 [01:06<00:00,  1.90s/it]


In [14]:
for key,value in dictionary_zro2.items():
    
    value = value[0]
    if value[0] == 'w': geometry = 'wire'
    elif value[0] == 's': geometry = 'slab'
    elif value[0] == 'b': geometry = 'bulk'
    elif value[0] == 'n': geometry = 'neck'
    elif value[0] == 'p': geometry = 'particle'
    else: geometry = 'unknown'

    if geometry == 'unknown': print(key)

    aux = dictionary_zro2[key][1]
    dictionary_zro2[key][1] = geometry
    dictionary_zro2[key].append(aux)

In [15]:
print(set([i[1] for i in dictionary_zro2.values()]))

{'wire', 'bulk', 'neck', 'particle', 'slab'}


In [17]:
np.save('dictionary_zro2.npy',dictionary_zro2)